This code converts a set of binary masks of 256x256 size into COCO format annotations, which is a JSON file that stores the information about the objects in the images and the relationships between the images and objects. The binary masks are assumed to be stored in the 'masks' folder and are processed one by one. For each mask image, its contours are found and stored as a list of points in the 'segmentation' field of an annotation. The annotations also include information about the image size, object category, and object bounding box. The resulting annotations are saved in a JSON file called 'annotations.json'.

In [15]:
import cv2
import numpy as np
import json
import os
from tqdm import tqdm

# Define the path to the folder containing the masks
mask_folder = 'C:\\Users\\path\\to\\masks\\forlder'
# Load all of the masks in the folder
mask_files = [f for f in os.listdir(mask_folder) if f.endswith('.jpg')]

# Initialize the COCO format dataset
coco_output = {
    "licenses": [
        {
            "name": "", 
            "id": 0, 
            "url": ""
        }
    ],
    "info": {
        "contributor": "", 
        "date_created": "", 
        "description": "", 
        "url": "", 
        "version": "", 
        "year": ""
    },
    "categories": [
        {
            "id": 1, 
            "name": "Solar_Panel", 
            "supercategory": ""
        }
    ],
    "images": [],
    "annotations": []
}

image_id = 1
annotation_id = 1

print("--Processing images:")
# Iterate over all of the masks
for i, mask_file in tqdm(enumerate(mask_files), total=len(mask_files)):
    # Iterate over all of the masks, Load the mask image and convert it to binary format, Find the contours of the objects in the binary mask
    mask = cv2.imread(os.path.join(mask_folder, mask_file), cv2.IMREAD_GRAYSCALE)
    _, binary_mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)
    
    # Find the contours of the objects in the binary mask
    contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Get the base file name without the "_label" suffix
    base_filename, _ = os.path.splitext(mask_file)
    base_filename = base_filename.replace("_label", "")
    
    # Store the image information
    image = {
        "id": image_id, 
        "width": 256,
        "height": 256,
        "file_name": f"{base_filename}.jpg",
        "license": 0,
        "flickr_url": "",
        "coco_url": "",
        "date_captured": 0
    }
    coco_output["images"].append(image)
    for j, contour in enumerate(contours):
        contour = contour.flatten().tolist()
        x = [point for point in contour[0::2]]
        y = [point for point in contour[1::2]]
        if len(x) < 2:
            continue
        segmentation = []
        for i in range(len(x)):
            segmentation.extend([x[i], y[i]])
        annotation = {
            "id": annotation_id,
            "image_id": image_id,
            "category_id": 1,
            "segmentation": [segmentation],
            "bbox": [min(x), min(y), max(x) - min(x), max(y) - min(y)],
            "iscrowd": 0
        }
        coco_output["annotations"].append(annotation)
        #print(annotation)

    image_id=image_id+1
    annotation_id=annotation_id+1

# Save the COCO format dataset to a JSON file
print("Exporting to annotations551.json.")
with open('annotations551.json', 'w') as fp:
    json.dump(coco_output, fp, indent=4, separators=(',', ': '))
    #json.dump(coco_output, fp)

--Processing images:


100%|██████████| 138/138 [00:00<00:00, 2313.29it/s]

Exporting to annotations551.json.


In [9]:
import json

def find_empty_or_small_segmentations(file_path):
    with open(file_path, "r") as f:
        data = json.load(f)

    empty_or_small_segmentations = []
    for image in data["images"]:
        for annotation in data["annotations"]:
            if image["id"] == annotation["image_id"] and len(annotation["segmentation"]) < 2:
                empty_or_small_segmentations.append((image["file_name"], annotation))

    return empty_or_small_segmentations

file_path = "masks/annotations.json"
empty_or_small_segmentations = find_empty_or_small_segmentations(file_path)
print("Number of empty or small segmentations:", len(empty_or_small_segmentations))
for file_name, annotation in empty_or_small_segmentations:
    print("File name:", file_name)

Number of empty or small segmentations: 2
File name: PV01_325122_1203837.jpg
File name: PV01_325122_1203837.jpg
